In [80]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [81]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [82]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [83]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [84]:
sallyMetrics = loadMetricData('sallyKung', "metrics")
sallyMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-09 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-09 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-09 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-09 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-09 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
sallyMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [86]:
# clean metric data

sallyMetrics.timestamp = pd.to_datetime(sallyMetrics.timestamp)

sallyMetrics.heartrate = sallyMetrics.heartrate.fillna(0)
sallyMetrics.steps = sallyMetrics.steps.fillna(0)
sallyMetrics.calories = sallyMetrics.calories.fillna(0)
sallyMetrics.gsr = sallyMetrics.gsr.fillna(0)
sallyMetrics.skintemp = sallyMetrics.skintemp.fillna(0)
sallyMetrics.airtemp = sallyMetrics.airtemp.fillna(0)

sallyMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-05-02 00:00:00,71,0,0.9,0.000056,91.4,88.7
1,2016-05-02 00:01:00,71,0,1.1,0.000056,91.4,89.6
2,2016-05-02 00:02:00,68,0,1.0,0.000058,91.4,90.5
3,2016-05-02 00:03:00,68,0,1.0,0.000058,92.3,91.4
4,2016-05-02 00:04:00,70,0,1.1,0.000060,92.3,91.4


In [87]:
sallyActivities = loadMetricData('sallyKung', "activities")
sallyActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
6,2016-05-05 00:48:54,2016-05-05T00:48:54Z,America/New_York,-240,2016-05-05 00:55:25,2016-05-05T00:55:25Z,America/New_York,-240,walk,311,725,14.000000,6.516667,103.729904,NaN,NaN,complete,1,walk__2016-05-05T004854
0,2016-05-05 14:03:04,2016-05-05T14:03:04Z,America/New_York,-240,2016-05-05 14:06:34,2016-05-05T14:06:34Z,America/New_York,-240,bike,210,NaN,5.700000,3.500000,101.328575,NaN,NaN,complete,1,bike__2016-05-05T140304
1,2016-05-05 15:30:28,2016-05-05T15:30:28Z,America/New_York,-240,2016-05-05 15:39:36,2016-05-05T15:39:36Z,America/New_York,-240,walk,197,503,12.800000,9.133333,113.649750,NaN,NaN,complete,1,walk__2016-05-05T153028
2,2016-05-05 15:42:03,2016-05-05T15:42:03Z,America/New_York,-240,2016-05-05 15:57:43,2016-05-05T15:57:43Z,America/New_York,-240,walk,565,1235,42.299999,15.666667,121.116814,NaN,NaN,complete,1,walk__2016-05-05T154203
3,2016-05-05 20:55:25,2016-05-05T20:55:25Z,America/New_York,-240,2016-05-05 21:00:53,2016-05-05T21:00:53Z,America/New_York,-240,walk,189,472,11.900000,5.466667,113.640213,NaN,NaN,complete,1,walk__2016-05-05T205525


In [88]:
sallySleep = loadMetricData('sallyKung', "sleep")
sallySleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-03 03:43:04,2016-05-03T03:43:04Z,America/New_York,-240,2016-05-03 10:17:04,2016-05-03T10:17:04Z,America/New_York,-240,200,77,...,36,sleep,22980,362.299988,70.224983,NaN,NaN,complete,2,sleep__2016-05-03T034304
0,2016-05-04 03:35:37,2016-05-04T03:35:37Z,America/New_York,-240,2016-05-04 11:30:37,2016-05-04T11:30:37Z,America/New_York,-240,223,148,...,34,sleep,28500,448.600006,69.207352,NaN,NaN,complete,2,sleep__2016-05-04T033537
0,2016-05-04 11:50:37,2016-05-04T11:50:37Z,America/New_York,-240,2016-05-04 12:22:37,2016-05-04T12:22:37Z,America/New_York,-240,31,0,...,1,sleep,1920,30.400000,70.055222,NaN,NaN,complete,2,sleep__2016-05-04T115037
1,2016-05-05 03:22:15,2016-05-05T03:22:15Z,America/New_York,-240,2016-05-05 11:46:15,2016-05-05T11:46:15Z,America/New_York,-240,249,145,...,36,sleep,30240,477.799988,67.988586,NaN,NaN,complete,2,sleep__2016-05-05T032215


In [89]:
sallySleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [90]:
sallyJournal = loadMetricData('sallyKung', "journal")
sallyJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
41,5/6/16 12:49,5/6/16 13:14,5/6/16,12:49 PM,1:14 PM,Orals,"79 5th Ave, 16th floor",Anxious,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,5/6/16 13:14,5/6/16 13:30,5/6/16,1:14 PM,1:30 PM,Talking to Abby,"79 5th Ave, 16th floor",Calm,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,5/6/16 13:47,5/6/16 15:00,5/6/16,1:47 PM,3:00 PM,With Sarah and Hashem,Durst,Angry,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,5/6/16 15:10,5/6/16 15:30,5/6/16,3:10 PM,3:30 PM,Eating fish and chips with Sarah,"66 12th Street, 407",Happy,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,5/6/16 16:21,5/6/16 16:21,5/6/16,4:21 PM,4:21 PM,Sarah messages me on Google,"66 12th Street, 407",Annoyed,1,NaN,NaN,NaN,NaN,1,NaN,1,NaN,1


In [91]:
sallyJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative      int64
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [92]:
# data= barbJournal.iloc[:0]
# clean metric data
sallyJournal["starttime"]= pd.to_datetime(sallyJournal.starttime)

# data= barbJournal.iloc[:1]
sallyJournal["endtime"]= pd.to_datetime(sallyJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

sallyJournal.qualitative = sallyJournal.qualitative.fillna(0)
sallyJournal.excited = sallyJournal.excited.fillna(0)
sallyJournal.happy = sallyJournal.happy.fillna(0)
sallyJournal.calm = sallyJournal.calm.fillna(0)
sallyJournal.anxious = sallyJournal.anxious.fillna(0)
sallyJournal.sad = sallyJournal.sad.fillna(0)
sallyJournal.angry = sallyJournal.angry.fillna(0)
sallyJournal.hungry = sallyJournal.hungry.fillna(0)
sallyJournal.tired = sallyJournal.tired.fillna(0)
sallyJournal.bored = sallyJournal.bored.fillna(0)

print(sallyJournal.shape)
sallyJournal.head(5)
# sallyJournal.tail(5)

(46, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,8,0,0,0,0,0,0,0
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,0,0,0,0,0,0,0,0
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,10,0,0,0,0,0,0,0
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,0,0,5,0,6,0,0,0
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,0,0,9,0,0,0,0,0


In [93]:

sallyJournal['boolexcited'] =  sallyJournal['excited'] > 0
sallyJournal['boolhappy'] =  sallyJournal['happy'] > 0
sallyJournal['boolcalm'] =  sallyJournal['calm'] > 0
sallyJournal['boolanxious'] =  sallyJournal['anxious'] > 0
sallyJournal['boolsad'] =  sallyJournal['sad'] > 0
sallyJournal['boolangry'] =  sallyJournal['angry'] > 0
sallyJournal['booltired'] =  sallyJournal['tired'] > 0
sallyJournal['boolhungry'] =  sallyJournal['hungry'] > 0
sallyJournal['boolbored'] =  sallyJournal['bored'] > 0


for i in range(0, sallyJournal.shape[0]):
#     sallyJournal['intqual'] = int(sallyJournal.iloc[i, 8])
    sallyJournal['intexcited'] = int(sallyJournal.iloc[i, 9])
    sallyJournal['inthappy'] = int(sallyJournal.iloc[i, 10]) 
    sallyJournal['intcalm'] = int(sallyJournal.iloc[i, 11])
    sallyJournal['intanxious'] = int(sallyJournal.iloc[i, 12])
    sallyJournal['intsad'] = int(sallyJournal.iloc[i, 13]) 
    sallyJournal['intangry'] = int(sallyJournal.iloc[i, 14]) 
    sallyJournal['inttired'] = int(sallyJournal.iloc[i, 15]) 
    sallyJournal['inthungry'] = int(sallyJournal.iloc[i, 16]) 
    sallyJournal['intbored'] = int(sallyJournal.iloc[i, 17]) 
    
    
sallyJournal['exercise'] = sallyJournal['activity'].str.contains('exercise')
sallyJournal['thesis'] = sallyJournal['activity'].str.contains('thesis')
sallyJournal['sarah'] = sallyJournal['activity'].str.contains('sarah')
sallyJournal['barb'] = sallyJournal['activity'].str.contains('barb')
sallyJournal['sol'] = sallyJournal['activity'].str.contains('sol')
sallyJournal['arash'] = sallyJournal['activity'].str.contains('arash')

sallyJournal['home'] = sallyJournal['location'].str.contains('home')
sallyJournal['work'] = sallyJournal['location'].str.contains("barb's office")







boolexcited = sallyJournal.iloc[:, 18].values.reshape(sallyJournal.shape[0], 1)
boolhappy = sallyJournal.iloc[:, 19].values.reshape(sallyJournal.shape[0], 1)
boolcalm = sallyJournal.iloc[:, 20].values.reshape(sallyJournal.shape[0], 1)
boolanxious = sallyJournal.iloc[:, 21].values.reshape(sallyJournal.shape[0], 1)
boolsad = sallyJournal.iloc[:, 22].values.reshape(sallyJournal.shape[0], 1)
boolangry = sallyJournal.iloc[:, 23].values.reshape(sallyJournal.shape[0], 1)
booltired = sallyJournal.iloc[:, 24].values.reshape(sallyJournal.shape[0], 1)
boolhungry = sallyJournal.iloc[:, 25].values.reshape(sallyJournal.shape[0], 1)
boolbored = sallyJournal.iloc[:, 26].values.reshape(sallyJournal.shape[0], 1)
intexcited = sallyJournal.iloc[:, 27].values.reshape(sallyJournal.shape[0], 1)
inthappy = sallyJournal.iloc[:, 28].values.reshape(sallyJournal.shape[0], 1)
intcalm = sallyJournal.iloc[:, 29].values.reshape(sallyJournal.shape[0], 1)
intanxious = sallyJournal.iloc[:, 30].values.reshape(sallyJournal.shape[0], 1)
intsad = sallyJournal.iloc[:, 31].values.reshape(sallyJournal.shape[0], 1)
intangry = sallyJournal.iloc[:, 32].values.reshape(sallyJournal.shape[0], 1)
inttired = sallyJournal.iloc[:, 33].values.reshape(sallyJournal.shape[0], 1)
inthungry = sallyJournal.iloc[:, 34].values.reshape(sallyJournal.shape[0], 1)
intbored = sallyJournal.iloc[:, 35].values.reshape(sallyJournal.shape[0], 1)
exercise = sallyJournal.iloc[:, 36].values.reshape(sallyJournal.shape[0], 1)
thesis = sallyJournal.iloc[:, 37].values.reshape(sallyJournal.shape[0], 1)
sarah = sallyJournal.iloc[:, 38].values.reshape(sallyJournal.shape[0], 1)
barb = sallyJournal.iloc[:, 39].values.reshape(sallyJournal.shape[0], 1)
sol = sallyJournal.iloc[:, 40].values.reshape(sallyJournal.shape[0], 1)
arash = sallyJournal.iloc[:, 41].values.reshape(sallyJournal.shape[0], 1)
work = sallyJournal.iloc[:, 42].values.reshape(sallyJournal.shape[0], 1)
home = sallyJournal.iloc[:, 43].values.reshape(sallyJournal.shape[0], 1)

sallyJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,inthungry,intbored,exercise,thesis,sarah,barb,sol,arash,home,work
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,...,0,1,False,False,False,False,False,False,False,False
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,...,0,1,False,False,False,False,False,False,False,False
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,...,0,1,False,False,False,False,False,False,False,False
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,...,0,1,False,False,False,False,False,False,False,False
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,...,0,1,False,False,False,False,False,False,False,False


In [94]:
# barbJournal.dtypes

In [95]:
# barbJournal.excited.convert_objects(convert_numeric=True)

In [96]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [97]:
# indicator for walking
totalData = makeIndicators('excitement', sallyMetrics, sallyJournal, 0, 1, 18)
totalData = makeIndicators('happy', sallyMetrics, sallyJournal, 0, 1, 19)
totalData = makeIndicators('calm', sallyMetrics, sallyJournal, 0, 1, 20)
totalData = makeIndicators('anxious', sallyMetrics, sallyJournal, 0, 1, 21)
totalData = makeIndicators('sad', sallyMetrics, sallyJournal, 0, 1, 22)
totalData = makeIndicators('angry', sallyMetrics, sallyJournal, 0, 1, 23)
totalData = makeIndicators('hungry', sallyMetrics, sallyJournal, 0, 1, 24)
totalData = makeIndicators('tired', sallyMetrics, sallyJournal, 0, 1, 25)
totalData = makeIndicators('bored', sallyMetrics, sallyJournal, 0, 1, 26)

totalData = makeIndicators('exercise', sallyMetrics, sallyJournal, 0, 1, 36)
totalData = makeIndicators('thesis', sallyMetrics, sallyJournal, 0, 1, 37)
totalData = makeIndicators('sarah', sallyMetrics, sallyJournal, 0, 1, 38)
totalData = makeIndicators('barb', sallyMetrics, sallyJournal, 0, 1, 39)
totalData = makeIndicators('sol', sallyMetrics, sallyJournal, 0, 1, 40)
totalData = makeIndicators('arash', sallyMetrics, sallyJournal, 0, 1, 41)
totalData = makeIndicators('work', sallyMetrics, sallyJournal, 0, 1, 42)
totalData = makeIndicators('home', sallyMetrics, sallyJournal, 0, 1, 43)

In [98]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  11520.000000  11520.000000  11520.000000  11520.000000  11520.000000   
mean      38.592101      4.214844      0.689028      0.007861     39.730790   
std       44.972087     19.327125      1.820190      0.171228     44.611131   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%       81.000000      0.000000      1.000000      0.000063     88.700000   
max      160.000000    695.000000    163.200000      6.650000     97.700000   

            airtemp   excitement       happy       calm    anxious  ...    \
count  11520.000000        11520       11520      11520      11520  ...     
mean      38.305477  0.000434028  0.00190972  0.0237847  0.0703125  ...     
std       43.080236    0.0208297   0.0436605   0.152384  

In [99]:
print(totalData.head())

            timestamp  heartrate  steps  calories       gsr  skintemp  \
0 2016-05-02 00:00:00         71      0       0.9  0.000056      91.4   
1 2016-05-02 00:01:00         71      0       1.1  0.000056      91.4   
2 2016-05-02 00:02:00         68      0       1.0  0.000058      91.4   
3 2016-05-02 00:03:00         68      0       1.0  0.000058      92.3   
4 2016-05-02 00:04:00         70      0       1.1  0.000060      92.3   

   airtemp excitement  happy   calm  ...    tired  bored exercise thesis  \
0     88.7      False  False  False  ...    False  False    False  False   
1     89.6      False  False  False  ...    False  False    False  False   
2     90.5      False  False  False  ...    False  False    False  False   
3     91.4      False  False  False  ...    False  False    False  False   
4     91.4      False  False  False  ...    False  False    False  False   

   sarah   barb    sol  arash   work   home  
0  False  False  False  False  False  False  
1  False  Fa

In [100]:
totalData.to_csv('totsally.csv')

In [101]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,tired,bored,exercise,thesis,sarah,barb,sol,arash,work,home
0,2016-05-02 00:00:00,71,0,0.9,0.000056,91.4,88.7,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-05-02 00:01:00,71,0,1.1,0.000056,91.4,89.6,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-05-02 00:02:00,68,0,1.0,0.000058,91.4,90.5,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-05-02 00:03:00,68,0,1.0,0.000058,92.3,91.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-05-02 00:04:00,70,0,1.1,0.000060,92.3,91.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [102]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(11520, 28)
            timestamp  heartrate  steps  calories       gsr  skintemp  \
0 2016-05-02 00:00:00         71      0       0.9  0.000056      91.4   
1 2016-05-02 00:01:00         71      0       1.1  0.000056      91.4   
2 2016-05-02 00:02:00         68      0       1.0  0.000058      91.4   
3 2016-05-02 00:03:00         68      0       1.0  0.000058      92.3   
4 2016-05-02 00:04:00         70      0       1.1  0.000060      92.3   

   airtemp excitement  happy   calm ...  sarah   barb    sol  arash   work  \
0     88.7      False  False  False ...  False  False  False  False  False   
1     89.6      False  False  False ...  False  False  False  False  False   
2     90.5      False  False  False ...  False  False  False  False  False   
3     91.4      False  False  False ...  False  False  False  False  False   
4     91.4      False  False  False ...  False  False  False  False  False   

    home weekday month hour min  
0  False       0     5    0   0  
1  False    

In [103]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
sarah = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)

In [104]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
        exercise, 
        thesis, 
        sarah, 
        barb, 
        sol, 
        arash, 
        work, 
        home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(11520, 17)

In [105]:
# define y
y = totalData.excitement.values
y.shape

(11520,)

In [106]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [107]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [108]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [109]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000496031746032 0.000289351851852
 Accuracy Rate    0.999503968254    0.999710648148
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00028935185185185184, 0.99971064814814814, 0.0, 1.0)

In [110]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000496031746032 0.000289351851852
      Accuracy Rate    0.999503968254    0.999710648148
     True Positives               0.0               0.0
     True Negatives                --               1.0


(0.00028935185185185184, 0.99971064814814814, 0.0, 1.0)

In [111]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00322420634921 0.00376157407407
 Accuracy Rate   0.996775793651   0.996238425926
True Positives              0.0              0.0
True Negatives               --   0.996526772793


(0.0037615740740740739, 0.99623842592592593, 0.0, 0.99652677279305357)

In [112]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000496031746032 0.000289351851852
 Accuracy Rate    0.999503968254    0.999710648148
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00028935185185185184, 0.99971064814814814, 0.0, 1.0)

In [113]:
# define y
y = totalData.happy.values
y.shape

(11520,)

In [114]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [115]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [116]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00210813492063 0.00144675925926
 Accuracy Rate   0.997891865079   0.998553240741
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0014467592592592592, 0.9985532407407407, 0.0, 1.0)

In [117]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00210813492063 0.00144675925926
      Accuracy Rate   0.997891865079   0.998553240741
     True Positives              0.0              0.0
     True Negatives               --              1.0


(0.0014467592592592592, 0.9985532407407407, 0.0, 1.0)

In [118]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00347222222222 0.00231481481481
 Accuracy Rate   0.996527777778   0.997685185185
True Positives              0.0              0.0
True Negatives               --   0.999130686757


(0.0023148148148148147, 0.99768518518518523, 0.0, 0.99913068675746164)

In [119]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00210813492063 0.00144675925926
 Accuracy Rate   0.997891865079   0.998553240741
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0014467592592592592, 0.9985532407407407, 0.0, 1.0)

In [120]:
# define y
y = totalData.calm.values
y.shape

(11520,)

In [121]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [122]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [123]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate  0.030505952381 0.0350115740741
 Accuracy Rate  0.969494047619  0.964988425926
True Positives 0.0222222222222 0.0212765957447
True Negatives              --  0.991374182035


(0.035011574074074077,
 0.96498842592592593,
 0.021276595744680851,
 0.99137418203450323)

In [124]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set   
------------------- ---------------- ---------------
         Error Rate  0.0292658730159 0.0347222222222
      Accuracy Rate   0.970734126984  0.965277777778
     True Positives 0.00555555555556             0.0
     True Negatives               --  0.992266508031


(0.034722222222222224, 0.96527777777777779, 0.0, 0.99226650803093397)

In [125]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0419146825397 0.0434027777778
 Accuracy Rate   0.95808531746  0.956597222222
True Positives 0.0777777777778 0.0957446808511
True Negatives              --  0.980666270077


(0.043402777777777776,
 0.95659722222222221,
 0.095744680851063829,
 0.98066627007733487)

In [126]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0224454365079 0.0271990740741
 Accuracy Rate  0.977554563492  0.972800925926
True Positives             0.0             0.0
True Negatives              --             1.0


(0.027199074074074073, 0.97280092592592593, 0.0, 1.0)

In [127]:
# define y
y = totalData.anxious.values
y.shape

(11520,)

In [128]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [129]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [130]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0634920634921 0.0613425925926
 Accuracy Rate  0.936507936508  0.938657407407
True Positives  0.129824561404          0.1375
True Negatives              --  0.998445273632


(0.061342592592592594,
 0.93865740740740744,
 0.13750000000000001,
 0.99844527363184077)

In [131]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0570436507937 0.0535300925926
      Accuracy Rate  0.942956349206  0.946469907407
     True Positives  0.357894736842  0.345833333333
     True Negatives              --  0.991293532338


(0.053530092592592594,
 0.94646990740740744,
 0.34583333333333333,
 0.99129353233830841)

In [132]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate  0.108630952381  0.105902777778
 Accuracy Rate  0.891369047619  0.894097222222
True Positives 0.0912280701754 0.0583333333333
True Negatives              --  0.956467661692


(0.10590277777777778,
 0.89409722222222221,
 0.058333333333333334,
 0.95646766169154229)

In [133]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0706845238095 0.0694444444444
 Accuracy Rate    0.92931547619  0.930555555556
True Positives 0.00350877192982             0.0
True Negatives               --             1.0


(0.069444444444444448, 0.93055555555555558, 0.0, 1.0)

In [134]:
# define y
y = totalData.sad.values
y.shape

(11520,)

In [135]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [136]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [137]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [138]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [139]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00297619047619 0.00347222222222
 Accuracy Rate   0.997023809524   0.996527777778
True Positives              0.0              nan
True Negatives               --   0.996527777778


(0.003472222222222222, 0.99652777777777779, nan, 0.99652777777777779)

In [140]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [141]:
# define y
y = totalData.angry.values
y.shape

(11520,)

In [142]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [143]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [144]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00496031746032 0.00318287037037
 Accuracy Rate    0.99503968254    0.99681712963
True Positives   0.121951219512   0.153846153846
True Negatives               --              1.0


(0.0031828703703703702, 0.99681712962962965, 0.15384615384615385, 1.0)

In [145]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00446428571429 0.00318287037037
      Accuracy Rate   0.995535714286    0.99681712963
     True Positives   0.121951219512   0.153846153846
     True Negatives               --              1.0


(0.0031828703703703702, 0.99681712962962965, 0.15384615384615385, 1.0)

In [146]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00458829365079 0.00318287037037
 Accuracy Rate   0.995411706349    0.99681712963
True Positives   0.121951219512   0.153846153846
True Negatives               --              1.0


(0.0031828703703703702, 0.99681712962962965, 0.15384615384615385, 1.0)

In [147]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00446428571429 0.00318287037037
 Accuracy Rate   0.995535714286    0.99681712963
True Positives   0.121951219512   0.153846153846
True Negatives               --              1.0


(0.0031828703703703702, 0.99681712962962965, 0.15384615384615385, 1.0)

In [148]:
# define y
y = totalData.hungry.values
y.shape

(11520,)

In [149]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [150]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [151]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.055183531746 0.0532407407407
 Accuracy Rate 0.944816468254  0.946759259259
True Positives 0.199255121043  0.203539823009
True Negatives             --  0.998761609907


(0.053240740740740741,
 0.9467592592592593,
 0.20353982300884957,
 0.99876160990712071)

In [152]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0657242063492  0.064525462963
      Accuracy Rate  0.934275793651  0.935474537037
     True Positives 0.0148975791434 0.0132743362832
     True Negatives              --             1.0


(0.064525462962962965, 0.93547453703703698, 0.013274336283185841, 1.0)

In [153]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0975942460317 0.0928819444444
 Accuracy Rate  0.902405753968  0.907118055556
True Positives 0.0558659217877 0.0309734513274
True Negatives              --  0.968421052632


(0.092881944444444448,
 0.90711805555555558,
 0.030973451327433628,
 0.96842105263157896)

In [154]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.138392857143 0.138599537037
 Accuracy Rate 0.861607142857 0.861400462963
True Positives 0.791433891993 0.756637168142
True Negatives             -- 0.868730650155


(0.13859953703703703,
 0.86140046296296302,
 0.75663716814159288,
 0.86873065015479878)

In [155]:
# define y
y = totalData.tired.values
y.shape

(11520,)

In [156]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [157]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [158]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [159]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [160]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00297619047619 0.00347222222222
 Accuracy Rate   0.997023809524   0.996527777778
True Positives              0.0              nan
True Negatives               --   0.996527777778


(0.003472222222222222, 0.99652777777777779, nan, 0.99652777777777779)

In [161]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [162]:
# define y
y = totalData.bored.values
y.shape

(11520,)

In [163]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [164]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [165]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [166]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [167]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00297619047619 0.00347222222222
 Accuracy Rate   0.997023809524   0.996527777778
True Positives              0.0              nan
True Negatives               --   0.996527777778


(0.003472222222222222, 0.99652777777777779, nan, 0.99652777777777779)

In [168]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)